# Final Notebook 1: Food Access in LA County
## Group Members: Madi Hamilton, Jessica Fay, Meaghan Woody, Branden Bohrnsen
### UP221 Winter 2024

### Description
**Research Question:** Are there geographic disparities trends in food insecurity in Los Angeles County?

**Notebook purpose:** Prepare data for creating maps

**Data sources:**
1. USC Neighborhood Data for Social Change
2. U.S. Census Bureau - American Community Survey 2016-2020
3. U.S. Census Tracts 2020
4. Grocery Store Walkability

### Description
**Research Question:** Are there geographic disparities trends in food insecurity in Los Angeles County?

**Notebook purpose:** Prepare data for creating maps

**Data sources:**
1. USC Neighborhood Data for Social Change
2. U.S. Census Bureau - American Community Survey 2016-2020
3. U.S. Census Tracts 2020
4. Grocery Store Walkability

### Import Libraries

In [8]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from libpysal.weights import Queen
import esda
from esda.moran import Moran, Moran_Local
import contextily as ctx
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster,plot_moran_simulation
import libpysal as lps
import matplotlib.pyplot as plt
import plotly.express as px
from shapely import wkt
from shapely.wkt import loads
import branca.colormap as cm
import folium

import numpy as np
import networkx as nx
import osmnx as ox
import contextily as ctx
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import plotly.express as px
from sodapy import Socrata
import mapclassify
from shapely.geometry import Point
import seaborn as sns

### Create merged dataset with variables and geometry

In [9]:
# Import merged dataset for USC and Census variables
food=pd.read_csv('finaldata_0303.csv')

In [10]:
# Load in tract shape file
tracts = gpd.read_file('tl_2020_06_tract.shp')
tracts['geoid20_x']=tracts.TRACTCE
tracts['COUNTYFP'].unique()

# Filter to LA County census tracts
latracts = tracts.query("COUNTYFP == '037'")

# Conditionally delete rows where hispanic % is outlier
new = food[food['% Hispanic or Latino'] <= 100]

In [11]:
# Merging the data
tracts_census= latracts.merge(new,on="geoid20_x")

In [ ]:
# Export DataFrame to CSV
tracts_census.to_csv('merged_data_jf.csv', index=False)

## Create new variables for analysis

In [6]:
# Create low access percent variable
a = food['lowaccess_count']
b =food['denom_total_pop']
food['lowaccess_pct']=round((a/b)*100)

In [7]:
# Create Food Insecurity Index 
# First, create a Natural Breaks classifier
classifier = mapclassify.NaturalBreaks.make(k=5)

# Age score
tracts_census['age_pct_score'] = tracts_census[['Percent 65 years and over']].apply(classifier)
tracts_census[['Percent 65 years and over', 'age_pct_score']].head()

# Hispanic score
tracts_census['hisp_pct_score'] = tracts_census[['% Hispanic or Latino']].apply(classifier)
tracts_census[['% Hispanic or Latino', 'hisp_pct_score']].head()

# Unemployment score
tracts_census['emp_pct_score'] = tracts_census[['% Unemployed']].apply(classifier)
tracts_census[['% Unemployed', 'emp_pct_score']].head()

# Food Insecurity Index: Age + Hispanic + Unemployment scores 
tracts_census['priority_index'] = tracts_census['age_pct_score']+tracts_census['hisp_pct_score']+tracts_census['emp_pct_score']

### Distribution of Work:

Jessica - focused on visualizations for descriptive analysis

Meaghan - focused on visualizations for descriptive analysis

Madi - focused on walkshed analysis

Branden - focused on folium analysis and spatial regression